In [ ]:
from diffusers import StableDiffusionInpaintPipeline
import torch
from PIL import Image
import numpy as np

# 加载 UNet 和 Text Encoder 的参数
with open("/content/unet_adapter.pth", "rb") as f:  # 使用正确的路径
    unet_params = torch.load(f)

with open("/content/text_encoder_adapter.pth", "rb") as f:  # 使用正确的路径
    text_encoder_params = torch.load(f)

# 使用预训练模型加载其余部分
pipe = StableDiffusionInpaintPipeline.from_pretrained("stablediffusionapi/cyberrealistic-41", torch_dtype=torch.float16)

# 替换 UNet 和 Text Encoder
pipe.unet.load_state_dict(unet_params, strict=False)  # 使用 strict=False 以允许部分加载
pipe.text_encoder.load_state_dict(text_encoder_params, strict=False)  # 同上

pipe.to("cuda")

# 加载你已经剪切掉3的图像
init_image = Image.open("dog1.png").convert("RGB")

# 创建一个蒙版图像，黑色表示缺失部分，白色表示保留部分
mask = Image.open("dog1.png").convert("L")
mask = np.array(mask)
mask[mask < 255] = 0
mask = Image.fromarray(mask)

# 使用修复管道进行图像修复
result = pipe(prompt="four corgis on the grass", image=init_image, mask_image=mask).images[0]

# 保存结果
result.save("result_dog1.png")



<ipython-input-24-1793877b3361>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unet_params = torch.load(f)
<ipython-input-24-1793877b3361>:11: FutureWarning: You are using

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--stablediffusionapi--cyberrealistic-41/snapshots/31259688a2398b11f5e7156bac475c459afaccd8/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--stablediffusionapi--cyberrealistic-41/snapshots/31259688a2398b11f5e7156bac475c459afaccd8/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--stablediffusionapi--cyberrealistic-41/snapshots/31259688a2398b11f5e7156bac475c459afaccd8/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--stablediffusionapi--cyberrealistic-41/snapshots/31259688a2398b11f5e7156bac475c459afaccd8/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
import json
import torch

# 读取 JSON 文件
with open("unet_adapter_config.json", "r") as f:
    json_data = json.load(f)

def convert_to_tensor(data):
    if isinstance(data, list):
        # 检查列表中的元素类型
        if all(isinstance(i, (int, float)) for i in data):
            return torch.tensor(data)
        else:
            return data  # 保留原列表
    elif isinstance(data, dict):
        return {key: convert_to_tensor(value) for key, value in data.items()}
    elif isinstance(data, (int, float)):  # 支持整数和浮点数
        return torch.tensor(data)
    else:
        # 对于字符串、布尔值等，直接返回原值
        return data

# 转换为张量
weights = convert_to_tensor(json_data)

# 保存为 .pth 文件
torch.save(weights, "unet_adapter.pth")


In [ ]:
import json
import torch

# 读取 JSON 文件
with open("text_encoder_adapter_config.json", "r") as f:
    json_data = json.load(f)

def convert_to_tensor(data):
    if isinstance(data, list):
        # 检查列表中的元素类型
        if all(isinstance(i, (int, float)) for i in data):
            return torch.tensor(data)
        else:
            return data  # 保留原列表
    elif isinstance(data, dict):
        return {key: convert_to_tensor(value) for key, value in data.items()}
    elif isinstance(data, (int, float)):  # 支持整数和浮点数
        return torch.tensor(data)
    else:
        # 对于字符串、布尔值等，直接返回原值
        return data

# 转换为张量
weights = convert_to_tensor(json_data)

# 保存为 .pth 文件
torch.save(weights, "text_encoder_adapter.pth")
